In [ ]:
import pandas as pd
import networkx as nx  
import numpy as np

In [ ]:
# Read all relationships
file = 'asso1140-1149.h5'

allrel = pd.read_hdf(file, key='asso', mode='r')

In [ ]:
# split relationships to positive (assodf) and negative (enemydf)
assodf = allrel[allrel.weight>0]
enemydf = allrel[allrel.weight<0]

# Calculate EVC

In [ ]:
# build a graph based on assodf
G = nx.from_pandas_edgelist(assodf, 'initiator', 'object', ['weight'])
# Calculate the eigenvector centrality (EVC) of assodf
centrality = nx.eigenvector_centrality(G ,max_iter=500, weight = 'weight' )
# Build a dataframe based on the eigenvector centrality 
evc = pd.DataFrame(data = centrality,index=[0]).T

In [ ]:
# save the eigenvector centrality to file
evc.to_csv('eigenvectorPositive.csv',header = None)

In [ ]:
evc.columns = ['originalEVC']
# calculate the lowest EVC
minEVC = min(evc.originalEVC)
# set the lowest EVC to ONE and then Logarithm
evc['transformedEV'] =  np.log(evc.originalEVC/minEVC)

# Calculae EnemyIndex

In [ ]:
# find the EVC of the object for each relationship
enemydf = enemydf.join(evc, on='object')
# weighted the relationship with EVC of the object 
enemydf['evcWeighted'] =  - enemydf.weight * enemydf.transformedEV
enemydf.drop(['weight','originalEVC','transformedEV'], axis = 1, inplace = True)

# add enemy index by initiator
enemyIndex = enemydf.groupby(['initiator'])['evcWeighted'].agg('sum')
enemyIndex.to_csv('enemyindex.csv')